# 09. 서포트 벡터 머신 (Support Vector Machine)

## 학습 목표
- SVM의 마진 최대화 원리 이해
- 서포트 벡터의 역할 학습
- 커널 트릭으로 비선형 문제 해결
- 하이퍼파라미터 C와 gamma 튜닝
- SVR로 회귀 문제 해결

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.svm import SVC, SVR, LinearSVC
from sklearn.datasets import (
    make_blobs, make_classification, make_moons, make_circles,
    load_iris, load_breast_cancer, load_diabetes
)
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)

## 1. 선형 SVM - 마진 최대화

SVM의 핵심은 두 클래스를 분리하는 최적의 초평면(hyperplane)을 찾는 것입니다.
마진(margin)을 최대화하여 일반화 성능을 높입니다.

In [ ]:
# 선형 분리 가능한 데이터 생성
X, y = make_blobs(n_samples=100, centers=2, random_state=6)

# 선형 SVM 학습
clf = svm.SVC(kernel='linear', C=1000)
clf.fit(X, y)

print(f"서포트 벡터 수: {len(clf.support_vectors_)}")
print(f"가중치 (w): {clf.coef_}")
print(f"절편 (b): {clf.intercept_}")

In [ ]:
# 결정 경계와 마진 시각화
plt.figure(figsize=(10, 8))

# 데이터 포인트
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', s=100, edgecolors='black')

# 결정 경계와 마진
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# 그리드 생성
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# 결정 경계와 마진 그리기
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1],
           linestyles=['--', '-', '--'], linewidths=[1, 2, 1])

# 서포트 벡터 표시
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
           s=200, linewidth=2, facecolors='none', edgecolors='green',
           label='Support Vectors')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Linear SVM: Maximum Margin Classifier')
plt.legend()
plt.show()

## 2. 소프트 마진 - C 파라미터

실제 데이터는 완벽하게 선형 분리가 불가능합니다.
C 파라미터로 오분류와 마진 크기의 균형을 조절합니다.

- **C 큼**: 오분류 페널티 큼 → 좁은 마진, 과적합 위험
- **C 작음**: 오분류 허용 → 넓은 마진, 일반화 향상

In [ ]:
# 노이즈가 있는 데이터
X, y = make_classification(
    n_samples=200, n_features=2, n_redundant=0,
    n_informative=2, n_clusters_per_class=1,
    flip_y=0.1,  # 10% 노이즈
    random_state=42
)

# 여러 C 값 비교
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
C_values = [0.1, 1, 100]

for ax, C in zip(axes, C_values):
    clf = svm.SVC(kernel='linear', C=C)
    clf.fit(X, y)

    # 결정 경계
    xlim = [X[:, 0].min() - 0.5, X[:, 0].max() + 0.5]
    ylim = [X[:, 1].min() - 0.5, X[:, 1].max() + 0.5]
    xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 100),
                         np.linspace(ylim[0], ylim[1], 100))

    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    ax.contour(xx, yy, Z, colors='k', levels=[-1, 0, 1],
               linestyles=['--', '-', '--'])
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='black')
    ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
               s=150, facecolors='none', edgecolors='green', linewidths=2)
    ax.set_title(f'C = {C}\nSupport Vectors: {len(clf.support_vectors_)}')

plt.tight_layout()
plt.show()

## 3. 커널 트릭 - 비선형 분류

커널 함수로 데이터를 고차원 공간에 매핑하여 비선형 패턴을 처리합니다.

주요 커널:
- **linear**: K(x, y) = x·y
- **polynomial**: K(x, y) = (γ·x·y + r)^d
- **rbf** (Gaussian): K(x, y) = exp(-γ||x - y||²)
- **sigmoid**: K(x, y) = tanh(γ·x·y + r)

In [ ]:
# 비선형 데이터 생성
X_moons, y_moons = make_moons(n_samples=200, noise=0.1, random_state=42)
X_circles, y_circles = make_circles(n_samples=200, noise=0.1, factor=0.5, random_state=42)

# 커널 비교
kernels = ['linear', 'poly', 'rbf']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for row, (X_data, y_data, name) in enumerate([(X_moons, y_moons, 'Moons'),
                                                (X_circles, y_circles, 'Circles')]):
    for col, kernel in enumerate(kernels):
        ax = axes[row, col]

        # SVM 학습
        if kernel == 'poly':
            clf = svm.SVC(kernel=kernel, degree=3, gamma='scale')
        else:
            clf = svm.SVC(kernel=kernel, gamma='scale')
        clf.fit(X_data, y_data)

        # 결정 경계
        xlim = [X_data[:, 0].min() - 0.5, X_data[:, 0].max() + 0.5]
        ylim = [X_data[:, 1].min() - 0.5, X_data[:, 1].max() + 0.5]
        xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 100),
                             np.linspace(ylim[0], ylim[1], 100))

        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        ax.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
        ax.scatter(X_data[:, 0], X_data[:, 1], c=y_data, cmap='coolwarm', edgecolors='black')
        ax.set_title(f'{name} - {kernel}\nAccuracy: {clf.score(X_data, y_data):.3f}')

plt.tight_layout()
plt.show()

## 4. RBF 커널과 gamma 파라미터

RBF 커널에서 gamma는 각 데이터 포인트의 영향 범위를 결정합니다.

- **gamma 큼**: 영향 범위 좁음 → 복잡한 경계, 과적합 위험
- **gamma 작음**: 영향 범위 넓음 → 단순한 경계, 과소적합 위험

In [ ]:
# gamma 효과 시각화
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
gamma_values = [0.1, 1, 10, 100]

X, y = make_moons(n_samples=200, noise=0.1, random_state=42)

for ax, gamma in zip(axes, gamma_values):
    clf = svm.SVC(kernel='rbf', gamma=gamma, C=1)
    clf.fit(X, y)

    xlim = [X[:, 0].min() - 0.5, X[:, 0].max() + 0.5]
    ylim = [X[:, 1].min() - 0.5, X[:, 1].max() + 0.5]
    xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 100),
                         np.linspace(ylim[0], ylim[1], 100))

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='black')
    ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
               s=100, facecolors='none', edgecolors='green', linewidths=2)
    ax.set_title(f'gamma = {gamma}\nSVs: {len(clf.support_vectors_)}')

plt.tight_layout()
plt.show()

## 5. SVC - 실제 데이터 분류

Iris 데이터셋으로 다중 클래스 분류를 수행합니다.
**중요**: SVM은 특성 스케일에 민감하므로 스케일링이 필수입니다.

In [ ]:
# 데이터 로드
iris = load_iris()
X, y = iris.data, iris.target

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 스케일링 (SVM은 스케일에 민감)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVM 학습
svm_clf = SVC(
    C=1.0,
    kernel='rbf',
    gamma='scale',
    probability=True,  # 확률 예측 활성화
    random_state=42
)
svm_clf.fit(X_train_scaled, y_train)

# 예측
y_pred = svm_clf.predict(X_test_scaled)

print("SVM 분류 결과:")
print(f"  정확도: {accuracy_score(y_test, y_pred):.4f}")
print(f"  서포트 벡터 수: {len(svm_clf.support_vectors_)}")
print("\n분류 리포트:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

In [ ]:
# 확률 예측
y_proba = svm_clf.predict_proba(X_test_scaled[:5])

print("확률 예측 (처음 5개):")
print(f"클래스: {iris.target_names}")
print(y_proba)
print(f"\n예측 클래스: {y_pred[:5]}")
print(f"실제 클래스: {y_test[:5]}")

## 6. 스케일링의 중요성

SVM은 거리 기반 알고리즘이므로 특성 스케일이 다르면 성능이 저하됩니다.

In [ ]:
# 스케일링 효과 비교
cancer = load_breast_cancer()
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    cancer.data, cancer.target, test_size=0.2, random_state=42
)

# 스케일링 없이
svm_no_scale = SVC(kernel='rbf', C=1, gamma='scale')
svm_no_scale.fit(X_train_c, y_train_c)
acc_no_scale = svm_no_scale.score(X_test_c, y_test_c)

# 스케일링 후
scaler = StandardScaler()
X_train_c_scaled = scaler.fit_transform(X_train_c)
X_test_c_scaled = scaler.transform(X_test_c)

svm_scaled = SVC(kernel='rbf', C=1, gamma='scale')
svm_scaled.fit(X_train_c_scaled, y_train_c)
acc_scaled = svm_scaled.score(X_test_c_scaled, y_test_c)

print("스케일링 효과:")
print(f"  스케일링 없이: {acc_no_scale:.4f}")
print(f"  스케일링 후:   {acc_scaled:.4f}")
print(f"  성능 향상:     {(acc_scaled - acc_no_scale) * 100:.2f}%")

## 7. 하이퍼파라미터 튜닝 - Grid Search

C와 gamma를 동시에 튜닝하여 최적 조합을 찾습니다.

In [ ]:
# 파라미터 그리드
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'kernel': ['rbf', 'poly']
}

# Grid Search
grid_search = GridSearchCV(
    SVC(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled, y_train)

print("\nGrid Search 결과:")
print(f"  최적 파라미터: {grid_search.best_params_}")
print(f"  최적 CV 점수: {grid_search.best_score_:.4f}")
print(f"  테스트 점수: {grid_search.score(X_test_scaled, y_test):.4f}")

In [ ]:
# C와 gamma 동시 튜닝 시각화 (RBF 커널만)
C_range = np.logspace(-2, 2, 5)
gamma_range = np.logspace(-3, 1, 5)

# 점수 계산
scores = np.zeros((len(C_range), len(gamma_range)))

for i, C in enumerate(C_range):
    for j, gamma in enumerate(gamma_range):
        svm_clf = SVC(C=C, gamma=gamma, kernel='rbf')
        svm_clf.fit(X_train_c_scaled, y_train_c)
        scores[i, j] = svm_clf.score(X_test_c_scaled, y_test_c)

# 히트맵 시각화
plt.figure(figsize=(10, 8))
plt.imshow(scores, interpolation='nearest', cmap='viridis')
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar(label='Accuracy')
plt.xticks(np.arange(len(gamma_range)), [f'{g:.3f}' for g in gamma_range])
plt.yticks(np.arange(len(C_range)), [f'{c:.2f}' for c in C_range])
plt.title('SVM Hyperparameter Tuning (RBF Kernel)')

# 최적점 표시
best_i, best_j = np.unravel_index(scores.argmax(), scores.shape)
plt.scatter(best_j, best_i, marker='*', s=300, c='red', edgecolors='white')

plt.tight_layout()
plt.show()

print(f"최적 C: {C_range[best_i]:.2f}")
print(f"최적 gamma: {gamma_range[best_j]:.3f}")
print(f"최고 정확도: {scores.max():.4f}")

## 8. SVR - Support Vector Regression

SVM을 회귀 문제에 적용합니다.
epsilon-tube 내의 오차는 무시하고, 튜브 밖의 오차만 페널티를 줍니다.

In [ ]:
# 데이터 로드
diabetes = load_diabetes()
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    diabetes.data, diabetes.target, test_size=0.2, random_state=42
)

# 스케일링
scaler = StandardScaler()
X_train_d_scaled = scaler.fit_transform(X_train_d)
X_test_d_scaled = scaler.transform(X_test_d)

# SVR 학습
svr = SVR(
    kernel='rbf',
    C=100,
    epsilon=0.1,  # 튜브 폭: 이 안의 오차는 무시
    gamma='scale'
)
svr.fit(X_train_d_scaled, y_train_d)

# 예측
y_pred_d = svr.predict(X_test_d_scaled)

print("SVR 회귀 결과:")
print(f"  MSE: {mean_squared_error(y_test_d, y_pred_d):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test_d, y_pred_d)):.4f}")
print(f"  R²: {r2_score(y_test_d, y_pred_d):.4f}")
print(f"  서포트 벡터 수: {len(svr.support_vectors_)}")

In [ ]:
# 시각화
plt.figure(figsize=(8, 6))
plt.scatter(y_test_d, y_pred_d, alpha=0.7, edgecolors='black')
plt.plot([y_test_d.min(), y_test_d.max()], [y_test_d.min(), y_test_d.max()], 'r--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title(f'SVR Regression (R² = {r2_score(y_test_d, y_pred_d):.4f})')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. 다중 클래스 분류 전략

SVM은 이진 분류기이므로 다중 클래스는 다음 전략으로 처리합니다.

- **OvO (One-vs-One)**: k(k-1)/2 개 분류기, SVC 기본값
- **OvR (One-vs-Rest)**: k 개 분류기, LinearSVC 기본값

In [ ]:
# OvO (기본)
svm_ovo = SVC(kernel='rbf', decision_function_shape='ovo')
svm_ovo.fit(X_train_scaled, y_train)
print(f"OvO 정확도: {svm_ovo.score(X_test_scaled, y_test):.4f}")

# OvR
svm_ovr = SVC(kernel='rbf', decision_function_shape='ovr')
svm_ovr.fit(X_train_scaled, y_train)
print(f"OvR 정확도: {svm_ovr.score(X_test_scaled, y_test):.4f}")

# LinearSVC (OvR 기본)
linear_svc = LinearSVC(dual=True, max_iter=10000)
linear_svc.fit(X_train_scaled, y_train)
print(f"LinearSVC 정확도: {linear_svc.score(X_test_scaled, y_test):.4f}")

## 10. 커널 비교 - 실전

유방암 데이터로 여러 커널의 성능을 비교합니다.

In [ ]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

print("커널별 성능 비교 (Breast Cancer):")
print("-" * 50)

for kernel in kernels:
    if kernel == 'poly':
        svm_model = SVC(kernel=kernel, degree=3, gamma='scale')
    else:
        svm_model = SVC(kernel=kernel, gamma='scale')

    svm_model.fit(X_train_c_scaled, y_train_c)
    acc = svm_model.score(X_test_c_scaled, y_test_c)
    print(f"  {kernel:8s}: {acc:.4f} (SVs: {len(svm_model.support_vectors_)})")

## 정리

### 핵심 개념

| 개념 | 설명 |
|------|------|
| **서포트 벡터** | 마진 경계에 위치한 핵심 데이터 포인트 |
| **마진** | 결정 경계와 서포트 벡터 사이의 거리 |
| **C** | 규제 파라미터 (큼: 좁은 마진, 작음: 넓은 마진) |
| **gamma** | RBF 커널 범위 (큼: 좁은 영향, 작음: 넓은 영향) |
| **커널** | 데이터를 고차원으로 매핑하는 함수 |

### SVM 사용 체크리스트

1. ✅ **스케일링 필수**: StandardScaler 또는 MinMaxScaler 적용
2. ✅ **커널 선택**: 선형 분리 가능 → linear, 비선형 → rbf
3. ✅ **파라미터 튜닝**: C와 gamma를 GridSearchCV로 튜닝
4. ✅ **대용량 데이터**: LinearSVC 또는 SGDClassifier 사용
5. ✅ **확률 필요시**: probability=True 설정 (추가 비용 발생)

### 장단점

**장점**:
- 고차원 데이터에 효과적
- 메모리 효율적 (서포트 벡터만 저장)
- 다양한 커널로 비선형 문제 해결

**단점**:
- 대용량 데이터에 느림 (O(n²) ~ O(n³))
- 스케일링 필수
- 파라미터 튜닝 필요

### 다음 단계
- k-Nearest Neighbors (kNN)
- Naive Bayes
- Ensemble methods